<a href="https://colab.research.google.com/github/shin-ta/Python-study/blob/main/%E3%82%BF%E3%82%A4%E3%82%BF%E3%83%8B%E3%83%83%E3%82%AF_20220420.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

前回勉強会では、勾配ブースティングで生存率を予測し、SIGNATEの評価が「0.8784929」（46位）となりました。

今回は、この評価の更新を目指します。

In [ ]:
# matplotlibの日本語対応ライブラリをインストール
!pip install japanize_matplotlib

**1. データを読み込もう**

In [ ]:
# アップロードしたデータをデータフレームに変換
import pandas as pd
train = pd.read_csv("train.tsv", sep="\t", index_col=0)                               # 学習用データ
test = pd.read_csv("test.tsv", sep="\t", index_col=0)                                 # 評価用データ
sample_submit = pd.read_csv("sample_submit.tsv", sep="\t", index_col=0, header=None)  # 応募用サンプルファイル

**2. データの概要を確認しよう**

In [ ]:
# 学習用データ、評価用データの先頭5件を確認
print(train.head())
print('====================================================================')
print(test.head())

In [ ]:
# 学習用データと評価用データのサイズ（行数、列数）を確認
train.shape, test.shape

In [ ]:
# 学習用データと評価用データの要約情報（データ型、欠損値有無 等）を確認
print(train.info())
print('====================================================================')
print(test.info())

In [ ]:
# 学習用データと評価用データの欠損値の割合を確認
print(train.isnull().sum()/train.count())
print('====================================================================')
print(test.isnull().sum()/test.count())

In [ ]:
# 学習用データと評価用データの基本統計量（数値項目）を確認
print(train[['pclass', 'age', 'sibsp', 'parch', 'fare', 'survived']].describe())
print('====================================================================')
print(test.describe())

In [ ]:
# 学習用データと評価用データの基本統計量（カテゴリ項目）を確認
category = ['sex', 'embarked']
print(train[category].describe())
print('====================================================================')
print(test[category].describe())

In [ ]:
# survivedと各項目の相関係数を取得
pd.get_dummies(train).corrwith(train["survived"])

In [ ]:
# 各項目間の相関係数を取得
pd.get_dummies(train).corr()

**3. データを分析してみよう**

In [ ]:
# ワーニングの出力を停止
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
# 各種モジュールをインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import japanize_matplotlib

In [ ]:
# 生存率を可視化
fig, ax = plt.subplots()
labels = ["0 : 死亡","1 : 生存"]
ax.pie(train['survived'].value_counts(), labels=labels ,autopct="%.1f%%")
ax.set_title('生存率')
plt.show()

In [ ]:
# 客室クラス別の人数と生存率を可視化
fig, ax = plt.subplots(ncols=3, figsize=(10,4), tight_layout=True)
sns.countplot(x='pclass', data=train, color='r', ax=ax[0])
sns.countplot(x='pclass', hue='survived', data=train, ax=ax[1])
sns.barplot(x='pclass', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('客室クラス別乗客数')
ax[1].set_title('客室クラス別死亡・生存者数')
ax[2].set_title('客室クラス別生存率')
plt.show()

In [ ]:
# 男女別の人数と生存率を可視化
fig, ax = plt.subplots(ncols=3, figsize=(10,4), tight_layout=True)
sns.countplot(x='sex', data=train, color='r', ax=ax[0])
sns.countplot(x='sex', hue='survived', data=train, ax=ax[1])
sns.barplot(x='sex', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('男女別乗客数')
ax[1].set_title('男女別死亡・生存者数')
ax[2].set_title('男女別生存率')
plt.show()

In [ ]:
# 乗船港別の人数と生存率を可視化
fig, ax = plt.subplots(ncols=3, figsize=(10,4), tight_layout=True)
sns.countplot(x='embarked', data=train, color='r', ax=ax[0])
sns.countplot(x='embarked', hue='survived', data=train, ax=ax[1])
sns.barplot(x='embarked', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('乗船港別乗客数')
ax[1].set_title('乗船港別死亡・生存者数')
ax[2].set_title('乗船港別生存率')
plt.show()

In [ ]:
# 年代別の人数と生存率を可視化
# 10歳毎の年代に分割
bins=list(range(0,81,10))
labels=['~'+str(x) for x in range(10, 81, 10)]
train['age_bin'] = pd.cut(train['age'], bins=bins, labels=labels)

fig, ax = plt.subplots(ncols=3, figsize=(10,4), tight_layout=True)
sns.countplot(x='age_bin', data=train, color='r', ax=ax[0])
sns.countplot(x='age_bin', hue='survived', data=train, ax=ax[1])
sns.barplot(x='age_bin', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('年代別乗客数')
ax[1].set_title('年代別死亡・生存者数')
ax[2].set_title('年代別生存率')
train = train.drop('age_bin', axis=1)
plt.show()


In [ ]:
# 運賃別の人数と生存率を可視化
# 10きざみの運賃に分割
bins=list(range(0,521,10))
bins[0]=-1
labels=['~'+str(x) for x in range(10, 521, 10)]
train['fare_bin'] = pd.cut(train['fare'], bins=bins, labels=labels)

fig, ax = plt.subplots(nrows=3, figsize=(12,8), tight_layout=True)
sns.countplot(x='fare_bin', data=train, color='r', ax=ax[0])
sns.countplot(x='fare_bin', hue='survived', data=train, ax=ax[1])
sns.barplot(x='fare_bin', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('運賃別乗客数')
ax[1].set_title('運賃別死亡・生存者数')
ax[2].set_title('運賃別生存率')
plt.setp(ax[0].get_xticklabels(), rotation=45)
plt.setp(ax[1].get_xticklabels(), rotation=45)
plt.setp(ax[2].get_xticklabels(), rotation=45)
train = train.drop('fare_bin', axis=1)
plt.show()

In [ ]:
# 家族数(兄弟、配偶者)別の人数と生存率を可視化
fig, ax = plt.subplots(ncols=3, figsize=(10,4), tight_layout=True)
sns.countplot(x='sibsp', data=train, color='r', ax=ax[0])
sns.countplot(x='sibsp', hue='survived', data=train, ax=ax[1])
sns.barplot(x='sibsp', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('家族数(兄弟、配偶者)別乗客数')
ax[1].set_title('家族数(兄弟、配偶者)別死亡・生存者数')
ax[2].set_title('家族数(兄弟、配偶者)別生存率')
plt.show()

In [ ]:
# 家族数(親、子供)別の人数と生存率を可視化
fig, ax = plt.subplots(ncols=3, figsize=(10,4), tight_layout=True)
sns.countplot(x='parch', data=train, color='r', ax=ax[0])
sns.countplot(x='parch', hue='survived', data=train, ax=ax[1])
sns.barplot(x='parch', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('家族数(親、子供)別乗客数')
ax[1].set_title('家族数(親、子供)別死亡・生存者数')
ax[2].set_title('家族数(親、子供)別生存率')
plt.show()

In [ ]:
# 総家族数別の人数と生存率を可視化
train['family'] = train['sibsp'] + train['parch']
fig, ax = plt.subplots(ncols=3, figsize=(10,4), tight_layout=True)
sns.countplot(x='family', data=train, color='r', ax=ax[0])
sns.countplot(x='family', hue='survived', data=train, ax=ax[1])
sns.barplot(x='family', y='survived', data=train, ci=None, color='b', ax=ax[2])
ax[0].set_title('総家族数別乗客数')
ax[1].set_title('総家族数別死亡・生存者数')
ax[2].set_title('総家族数別生存率')
train = train.drop('family', axis=1)
plt.show()

In [ ]:
# survivedと各項目の相関係数を可視化
corr = pd.get_dummies(train).corrwith(train["survived"])

fig, ax = plt.subplots()
ax.barh(corr.index, corr, color='r')
ax.set_title('「survived」と各項目の相関係数')
ax.set_xlim(-1, 1)
ax.grid(axis='x')

In [ ]:
# 各項目間の相関係数をヒートマップで可視化
corr = pd.get_dummies(train).corr()

fig, ax = plt.subplots(figsize=(15,4))
sns.heatmap(corr, cmap= sns.color_palette('coolwarm', 10), annot=True, fmt='.4f', vmin = -1, vmax = 1, ax=ax)
ax.set_title('各項目間の相関係数')
plt.show()

In [ ]:
# 散布図行列を表示
sns.pairplot(data=train, hue="survived", dropna=True, size=2)

In [ ]:
# 年齢を男女別、客室クラス別、生存有無で層別化したカーネル密度推定グラフを可視化
oldest = train['age'].max()

facet1 = sns.FacetGrid(train, hue="sex", aspect=2)
facet1.map(sns.kdeplot, 'age', shade= True)
facet1.set(xlim=(0, oldest))
facet1.add_legend()

facet2 = sns.FacetGrid(train, hue="pclass",aspect=2)
facet2.map(sns.kdeplot,'age',shade= True)
facet2.set(xlim=(0, oldest))
facet2.add_legend()

facet3 = sns.FacetGrid(train, hue="survived",aspect=2)
facet3.map(sns.kdeplot,'age',shade= True)
facet3.set(xlim=(0, oldest))
facet3.add_legend()

In [ ]:
# 家族人数を生存有無で層別化したカーネル密度推定グラフを可視化
train['family'] = train['sibsp'] + train['parch']

facet1 = sns.FacetGrid(train, hue="survived",height=2, aspect=3, ylim=(0, 1.2))
facet1.map(sns.kdeplot,'family',shade= True)
facet1.set(xlim=(0, train['family'].max()))
facet1.add_legend()

facet1 = sns.FacetGrid(train, hue="survived",height=2, aspect=3, ylim=(0, 1.2))
facet1.map(sns.kdeplot,'sibsp',shade= True)
facet1.set(xlim=(0, train['family'].max()))
facet1.add_legend()

facet2 = sns.FacetGrid(train, hue="survived",height=2, aspect=3, ylim=(0, 1.2))
facet2.map(sns.kdeplot,'parch',shade= True)
facet2.set(xlim=(0, train['family'].max()))
facet2.add_legend()

**4. データの前処理をしてみよう**

In [ ]:
# 学習用データと検証用データを結合（一括で前処理を行うため）
data = pd.concat([train, test])

In [ ]:
# embarkedの欠損値処理を実行
data['embarked'] = data['embarked'].fillna(data['embarked'].mode()) 

In [ ]:
# sibspとparchを統合し新しい特徴量（family）を作成
data['family'] = data['sibsp'] + data['parch']
data = data.drop(['sibsp', 'parch'], axis=1)

In [ ]:
# カテゴリー値（sex、embarked）をダミー変数で数値化
data = pd.get_dummies(data)

In [ ]:
# 欠損値補完前のデータを確認
data.head(20)

In [ ]:
# ageの欠損値をランダムフォレストで予測した結果で補完
from sklearn.ensemble import RandomForestRegressor

# 予測に使用する項目でデータフレームを作成
age_data = data.drop('survived', axis=1)

# 学習データ（欠損なし）とテストデータ（欠損あり）に分割
train_age = age_data[age_data['age'].notnull()]
test_age = age_data[age_data['age'].isnull()].drop('age', axis=1)

# 学習用データを目的変数と説明変数に分割
y = train_age['age']
X = train_age.drop('age', axis=1)  

# ランダムフォレストで予測モデルを作成
clf = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
clf.fit(X, y)

# テストデータのageを予測
pred = clf.predict(test_age)
pred

# 欠損値を補完
data.loc[test_age.index, 'age'] = pred

In [ ]:
# 欠損値補完後のデータを確認
data.head(20)

In [ ]:
# 結合したデータを学習用データと検証用データに分割
train = data.loc[train.index]
test = data.loc[test.index]
test = test.drop(["survived"], axis=1)

In [ ]:
# 学習用データを目的変数と説明変数に分割
y = train["survived"]                 # 目的変数
X = train.drop(['survived'], axis=1)  # 目的変数を除いたデータ

**5. モデリングをしてみよう**

In [ ]:
# 学習用データをモデル訓練用とモデル検証用に分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# グリッドサーチで最良のパラメータの組み合わせを取得
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# パラメータ候補を辞書型で設定
param = {'max_depth':[2, 3, 4],                                             # デフォルト値：3
         'learning_rate':[0.024, 0.025, 0.026, 0.027],                      # デフォルト値：0.1
         'n_estimators':[103, 104, 105, 106, 107, 108, 109, 110, 111],      # デフォルト値：100
         'random_state':[0]}

# 勾配ブースティングによる学習
cv = 5
clf = GridSearchCV(GradientBoostingClassifier(), param, cv=cv)
clf.fit(X_train,y_train)                           
 
# 結果の確認
print('最良条件:\n', clf.best_estimator_)
print('最良スコア:\n',clf.best_score_)

In [ ]:
# 最良パラメータの勾配ブースティングでモデリング
model = GradientBoostingClassifier(learning_rate=0.026, 
                                   n_estimators=104,
                                   random_state=0)
model.fit(X_train, y_train)
print("訓練データによる評価: ",model.score(X_train, y_train))
print("検証データによる評価: ",model.score(X_test, y_test))

In [ ]:
# 評価用データの生存率を予測
pred = model.predict_proba(test)[:, 1] 

**6. 予測結果を投稿しよう**

In [ ]:
# 応募用サンプルファイルに生存率をセットして出力
sample_submit[1] = pred
sample_submit.to_csv('submit_mmdd_x.tsv', header=None, sep='\t')

**お疲れ様でした！**